### Import Modules and Python Functions

In [1]:
import os
import numpy as np
import pandas as pd
import yaml
with open('../python/variables.yaml') as info:
    VARS = yaml.load(info, Loader=yaml.FullLoader)

#os.path.join( os.path.dirname('__file__' ), '../python' )
#from from_csv_to_df import get_select_data

### Import Variables

In [23]:
# Events
events_csv = VARS['EVENTS_CSV']
events_cols_all = VARS['EVENTS_COLS_ALL']
events_cols_select = VARS['EVENTS_COLS_SELECT']

# Mentions
mentions_csv = VARS['MENTIONS_CSV']
mentions_cols_all = VARS['MENTIONS_COLS_ALL']
mentions_cols_select = VARS['MENTIONS_COLS_SELECT']

# Pilot
cameo_verbs = VARS['PILOT_CAMEO_VERB_CODES']
select_countries_60 = VARS['PILOT_COUNTRIES_IOS2']

### Define Reusable Python Functions

In [3]:
def get_select_data(filepath: str, all_cols: list, select_cols: list) -> object:
    """Get Select Columns of Data from GDELT Latest Update CSV
    :param filepath: full filename and path to CSV file to be imported
    :param all_cols: column names of the CSV imported
    :param select_cols: derivative list of columns from all_cols
    :rtype: dataframe
    :return: dataframe
    """

    # Import entire CSV
    latest_update_df = pd.DataFrame(pd.read_csv(filepath,
                                                names=all_cols,
                                                delimiter="\t"))

    # Select specific columns
    return latest_update_df[select_cols]

### Import Data

#### Events Data

In [5]:
# get data
e1_df = get_select_data(events_csv,
                        events_cols_all,
                        events_cols_select)

e2_df = get_select_data('../select_data/data/latest_gdelt_events_2.csv',
                        events_cols_all,
                        events_cols_select)

e3_df = get_select_data('../select_data/data/latest_gdelt_events_3.csv',
                        events_cols_all,
                        events_cols_select)

merge_e_df = pd.merge(e1_df, e2_df, on=events_cols_select, how='outer')
gdelt_se_df = pd.merge(merge_e_df, e3_df, on=events_cols_select, how='outer')
print(gdelt_se_df.shape)
print(gdelt_se_df.info())
gdelt_se_df.head(1)

(3822, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3822 entries, 0 to 3821
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GLOBALEVENTID          3822 non-null   int64  
 1   SQLDATE                3822 non-null   int64  
 2   EventCode              3822 non-null   int64  
 3   EventRootCode          3822 non-null   int64  
 4   QuadClass              3822 non-null   int64  
 5   GoldsteinScale         3822 non-null   float64
 6   ActionGeo_Type         3822 non-null   int64  
 7   ActionGeo_FullName     3716 non-null   object 
 8   ActionGeo_CountryCode  3719 non-null   object 
 9   ActionGeo_Lat          3716 non-null   float64
 10  ActionGeo_Long         3716 non-null   float64
 11  SOURCEURL              3822 non-null   object 
dtypes: float64(3), int64(6), object(3)
memory usage: 388.2+ KB
None


,GLOBALEVENTID,SQLDATE,EventCode,EventRootCode,QuadClass,GoldsteinScale,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
0,972238884,20200301,20,2,1,3.0,4,"Canberra, Australian Capital Territory, Australia",AS,-35.2833,149.217,https://www.aninews.in/news/world/asia/china-a...


In [6]:
print('Number of Global Event Ids: ', len(gdelt_se_df['GLOBALEVENTID'].unique()))

Number of Global Event Ids:  3822


In [7]:
# Assess date range
print('Min Event Date: ', gdelt_se_df['SQLDATE'].min())
print('Max Event Date: ', gdelt_se_df['SQLDATE'].max())

Min Event Date:  20200301
Max Event Date:  20210303


### Mentions Data

In [8]:
# get data
m1_df = get_select_data(mentions_csv,
                              mentions_cols_all,
                              mentions_cols_select)

m2_df = get_select_data('../select_data/data/latest_gdelt_mentions_2.csv',
                              mentions_cols_all,
                              mentions_cols_select)

m3_df = get_select_data('../select_data/data/latest_gdelt_mentions_3.csv',
                              mentions_cols_all,
                              mentions_cols_select)

merge_m_df = pd.merge(m1_df, m2_df, on=mentions_cols_select, how='outer')
gdelt_sm_df = pd.merge(merge_m_df, m2_df, on=mentions_cols_select, how='outer')
print(gdelt_sm_df.shape)
print(gdelt_sm_df.info())
gdelt_sm_df.head(1)

(4325, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4325 entries, 0 to 4324
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GLOBALEVENTID    4325 non-null   int64  
 1   EventTimeDate    4325 non-null   int64  
 2   MentionTimeDate  4325 non-null   int64  
 3   Confidence       4325 non-null   int64  
 4   MentionDocTone   4325 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 202.7 KB
None


,GLOBALEVENTID,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone
0,972238884,20210301204500,20210301204500,20,-6.52921


In [9]:
print('Number of Global Event Ids: ', len(gdelt_sm_df['GLOBALEVENTID'].unique()))

Number of Global Event Ids:  3229


In [11]:
# Assess date range
print('Min Mentions Date: ', gdelt_sm_df['MentionTimeDate'].min())
print('Max Mentions Date: ', gdelt_sm_df['MentionTimeDate'].max())

Min Mentions Date:  20210301204500
Max Mentions Date:  20210301204500


#### Select Mentions within first 60 Days of an Event

In [12]:
# Calculate days between
gdelt_sm_df['DaysBetween'] = gdelt_sm_df['MentionTimeDate'] - gdelt_sm_df['EventTimeDate']
#print(gdelt_sm_df['DaysBetween'][0])

# Drop rows where days_between <= 60 days
gdelt_sm_60d_df = gdelt_sm_df[gdelt_sm_df['DaysBetween'] <= 60].reset_index(drop=True)
print(gdelt_sm_60d_df.shape)
print('Number of Global Event Ids: ', len(gdelt_sm_60d_df['GLOBALEVENTID'].unique()))
gdelt_sm_60d_df.head()

(1324, 6)
Number of Global Event Ids:  1212


,GLOBALEVENTID,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone,DaysBetween
0,972238884,20210301204500,20210301204500,20,-6.529210,0
1,972238885,20210301204500,20210301204500,20,-6.529210,0
2,972238886,20210301204500,20210301204500,60,-6.529210,0
3,972238887,20210301204500,20210301204500,40,-4.385965,0
4,972238888,20210301204500,20210301204500,40,-4.385965,0


In [13]:
# Verify output
max(gdelt_sm_60d_df['DaysBetween'])

0

#### Change int64 dates to datetimes

In [14]:
date_format = '%Y%m%d%H%M%S'
gdelt_sm_60d_df['EventTimeDate'] = pd.to_datetime(gdelt_sm_60d_df['EventTimeDate'].astype(str), format=date_format)
gdelt_sm_60d_df['MentionTimeDate'] = pd.to_datetime(gdelt_sm_60d_df['MentionTimeDate'].astype(str), format=date_format)
gdelt_sm_60d_df.head()

,GLOBALEVENTID,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone,DaysBetween
0,972238884,2021-03-01 20:45:00,2021-03-01 20:45:00,20,-6.529210,0
1,972238885,2021-03-01 20:45:00,2021-03-01 20:45:00,20,-6.529210,0
2,972238886,2021-03-01 20:45:00,2021-03-01 20:45:00,60,-6.529210,0
3,972238887,2021-03-01 20:45:00,2021-03-01 20:45:00,40,-4.385965,0
4,972238888,2021-03-01 20:45:00,2021-03-01 20:45:00,40,-4.385965,0


#### Group Mentions Data by GlobalEventId for Average Confidence and Mention Tone

agg_cols = ['GLOBALEVENTID','MeanConfidence', 'MeanMentionDocTone']

gdelt_sm_60d_agg_df = gdelt_sm_60d_df.groupby(['GLOBALEVENTID', 'EventTimeDate'], as_index=False,
                                             ).mean() # get mean Confidence, Tone, DaysBetween
gdelt_sm_60d_agg_df.head()

### Join Events and Mentions Data on GlobalEventId

In [15]:
print('Events Data: ', gdelt_se_df.shape)
print('Mentions Data: ', gdelt_sm_60d_df.shape)

# Merge dataframes
merged_df = gdelt_se_df.merge(gdelt_sm_60d_df, how='left', on='GLOBALEVENTID')
print('Merged Data w/ Duplicates: ', merged_df.shape)

# Drop duplicates
merged_df = merged_df.drop_duplicates()
print('Merged Data w/o Duplicates: ',merged_df.shape)
merged_df.head()

Events Data:  (3822, 12)
Mentions Data:  (1324, 6)
Merged Data w/ Duplicates:  (3934, 17)
Merged Data w/o Duplicates:  (3911, 17)


,GLOBALEVENTID,SQLDATE,EventCode,EventRootCode,QuadClass,GoldsteinScale,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,SOURCEURL,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone,DaysBetween
0,972238884,20200301,20,2,1,3.0,4,"Canberra, Australian Capital Territory, Australia",AS,-35.2833,149.2170,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,20.0,-6.529210,0.0
1,972238885,20200301,20,2,1,3.0,4,"Beijing, Beijing, China",CH,39.9289,116.3880,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,20.0,-6.529210,0.0
2,972238886,20200301,20,2,1,3.0,4,"Beijing, Beijing, China",CH,39.9289,116.3880,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,60.0,-6.529210,0.0
3,972238887,20200301,111,11,3,-2.0,4,"Mumbai, Maharashtra, India",IN,18.9750,72.8258,https://www.techshout.com/security/2021/01/chi...,2021-03-01 20:45:00,2021-03-01 20:45:00,40.0,-4.385965,0.0
4,972238888,20200301,110,11,3,-2.0,4,"Mumbai, Maharashtra, India",IN,18.9750,72.8258,https://www.techshout.com/security/2021/01/chi...,2021-03-01 20:45:00,2021-03-01 20:45:00,40.0,-4.385965,0.0


In [16]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3911 entries, 0 to 3933
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GLOBALEVENTID          3911 non-null   int64         
 1   SQLDATE                3911 non-null   int64         
 2   EventCode              3911 non-null   int64         
 3   EventRootCode          3911 non-null   int64         
 4   QuadClass              3911 non-null   int64         
 5   GoldsteinScale         3911 non-null   float64       
 6   ActionGeo_Type         3911 non-null   int64         
 7   ActionGeo_FullName     3798 non-null   object        
 8   ActionGeo_CountryCode  3807 non-null   object        
 9   ActionGeo_Lat          3798 non-null   float64       
 10  ActionGeo_Long         3798 non-null   float64       
 11  SOURCEURL              3911 non-null   object        
 12  EventTimeDate          1301 non-null   datetime64[ns]
 13  Men

### Replace Cameo Code Root Integer Values with Associated String

In [24]:
cameo_root_code = list(merged_df['EventRootCode'].sort_values(ascending=True).unique())
cameo_root_code

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [22]:
print(cameo_verbs)

['MAKE PUBLIC STATEMENT', 'APPEAL', 'EXPRESS INTENT TO COOPERATE', 'CONSULT', 'ENGAGE IN DIPLOMATIC COOPERATION', 'ENGAGE IN MATERIAL COOPERATION', 'PROVIDE AID', 'YIELD', 'INVESTIGATE', 'DEMAND', 'DISAPPROVE', 'REJECT', 'THREATEN', 'PROTEST', 'EXHIBIT MILITARY POSTURE', 'REDUCE RELATIONS', 'COERCE', 'ASSAULT', 'FIGHT', 'ENGAGE IN UNCONVENTIONAL MASS VIOLENCE']


In [28]:
# Convert lists to dictionary 
cameo_code_dict = {cameo_root_code[i]: cameo_verbs[i] for i in range(len(cameo_root_code))}
cameo_code_dict

{1: 'MAKE PUBLIC STATEMENT',
 2: 'APPEAL',
 3: 'EXPRESS INTENT TO COOPERATE',
 4: 'CONSULT',
 5: 'ENGAGE IN DIPLOMATIC COOPERATION',
 6: 'ENGAGE IN MATERIAL COOPERATION',
 7: 'PROVIDE AID',
 8: 'YIELD',
 9: 'INVESTIGATE',
 10: 'DEMAND',
 11: 'DISAPPROVE',
 12: 'REJECT',
 13: 'THREATEN',
 14: 'PROTEST',
 15: 'EXHIBIT MILITARY POSTURE',
 16: 'REDUCE RELATIONS',
 17: 'COERCE',
 18: 'ASSAULT',
 19: 'FIGHT'}

In [32]:
# Add column for cameo code root strings (verbs)
merged_df['EventRootCodeString'] = merged_df['EventRootCode'].map(cameo_code_dict)

# verify output
cameo_code_df = merged_df[['EventRootCode', 'EventRootCodeString']].sort_values(by='EventRootCode',
                                                                                ascending=True).drop_duplicates()
cameo_code_df

,EventRootCode,EventRootCodeString
3933,1,MAKE PUBLIC STATEMENT
1380,2,APPEAL
2762,3,EXPRESS INTENT TO COOPERATE
2423,4,CONSULT
2453,5,ENGAGE IN DIPLOMATIC COOPERATION
1269,6,ENGAGE IN MATERIAL COOPERATION
1644,7,PROVIDE AID
1923,8,YIELD
1557,9,INVESTIGATE
3034,10,DEMAND


### Assess Null Values

In [33]:
null_df = merged_df[pd.isnull(merged_df).any(axis=1)]
print(null_df.shape)
print(null_df.info())
null_df.head()

(2652, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2652 entries, 22 to 3933
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GLOBALEVENTID          2652 non-null   int64         
 1   SQLDATE                2652 non-null   int64         
 2   EventCode              2652 non-null   int64         
 3   EventRootCode          2652 non-null   int64         
 4   QuadClass              2652 non-null   int64         
 5   GoldsteinScale         2652 non-null   float64       
 6   ActionGeo_Type         2652 non-null   int64         
 7   ActionGeo_FullName     2539 non-null   object        
 8   ActionGeo_CountryCode  2548 non-null   object        
 9   ActionGeo_Lat          2539 non-null   float64       
 10  ActionGeo_Long         2539 non-null   float64       
 11  SOURCEURL              2652 non-null   object        
 12  EventTimeDate          42 non-null     datetime64[

,GLOBALEVENTID,SQLDATE,EventCode,EventRootCode,QuadClass,GoldsteinScale,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,SOURCEURL,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone,DaysBetween,EventRootCodeString
22,972238906,20210130,10,1,1,0.0,0,NaN,NaN,NaN,NaN,http://quickworldnews.com/meghan-and-harry-dis...,2021-03-01 20:45:00,2021-03-01 20:45:00,50.0,3.225806,0.0,MAKE PUBLIC STATEMENT
23,972238907,20210130,51,5,1,3.4,0,NaN,NaN,NaN,NaN,http://quickworldnews.com/meghan-and-harry-dis...,2021-03-01 20:45:00,2021-03-01 20:45:00,50.0,3.225806,0.0,ENGAGE IN DIPLOMATIC COOPERATION
36,972238920,20210222,15,1,1,0.0,0,NaN,NaN,NaN,NaN,https://www.journalinquirer.com/crime_and_cour...,2021-03-01 20:45:00,2021-03-01 20:45:00,100.0,-8.502024,0.0,MAKE PUBLIC STATEMENT
40,972238924,20210222,10,1,1,0.0,0,NaN,NaN,NaN,NaN,https://www.kelownadailycourier.ca/news/articl...,2021-03-01 20:45:00,2021-03-01 20:45:00,50.0,-0.789474,0.0,MAKE PUBLIC STATEMENT
72,972238956,20210301,841,8,2,7.0,0,NaN,NaN,NaN,NaN,https://edmonton.ctvnews.ca/high-risk-sex-offe...,2021-03-01 20:45:00,2021-03-01 20:45:00,100.0,-3.982301,0.0,YIELD


#### Based on project requirements, the data source for visualization presence of *non-null* values in the following columns:
- GlobalEventId
- EventTimeDate
- ActionGeo_CountryCode
- EventCode
- GoldsteinScale
- MentionDocTone

In [34]:
# Drop all rows in merged_df with nulls in the specified columns
required_value_columns = ['GLOBALEVENTID', 'EventTimeDate', 'ActionGeo_CountryCode', 
                          'EventCode', 'GoldsteinScale', 'MentionDocTone']

cleaned_merged_df = merged_df[~pd.isnull(merged_df[required_value_columns]).any(axis=1)].reset_index(drop=True)
print(cleaned_merged_df.shape)
cleaned_merged_df.head()

(1267, 18)


,GLOBALEVENTID,SQLDATE,EventCode,EventRootCode,QuadClass,GoldsteinScale,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,SOURCEURL,EventTimeDate,MentionTimeDate,Confidence,MentionDocTone,DaysBetween,EventRootCodeString
0,972238884,20200301,20,2,1,3.0,4,"Canberra, Australian Capital Territory, Australia",AS,-35.2833,149.2170,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,20.0,-6.529210,0.0,APPEAL
1,972238885,20200301,20,2,1,3.0,4,"Beijing, Beijing, China",CH,39.9289,116.3880,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,20.0,-6.529210,0.0,APPEAL
2,972238886,20200301,20,2,1,3.0,4,"Beijing, Beijing, China",CH,39.9289,116.3880,https://www.aninews.in/news/world/asia/china-a...,2021-03-01 20:45:00,2021-03-01 20:45:00,60.0,-6.529210,0.0,APPEAL
3,972238887,20200301,111,11,3,-2.0,4,"Mumbai, Maharashtra, India",IN,18.9750,72.8258,https://www.techshout.com/security/2021/01/chi...,2021-03-01 20:45:00,2021-03-01 20:45:00,40.0,-4.385965,0.0,DISAPPROVE
4,972238888,20200301,110,11,3,-2.0,4,"Mumbai, Maharashtra, India",IN,18.9750,72.8258,https://www.techshout.com/security/2021/01/chi...,2021-03-01 20:45:00,2021-03-01 20:45:00,40.0,-4.385965,0.0,DISAPPROVE


In [65]:
# Store dataframe globally
%store cleaned_merged_df

Stored 'cleaned_merged_df' (DataFrame)
